In [1]:
import pandas as pd
import random
import wordninja

In [2]:
pos_locations = ['positive data/spring-boot_labeled_sentences.csv','positive data/facebook_labeled_sentences.csv',
                 'positive data/tensorflow_labeled_sentences.csv','positive data/vscode_labeled_sentences.csv',
                 'positive data/flutter_labeled_sentences.csv']
q_locations = ['data/spring-boot_PRs.txt','data/facebook_PRs.txt',
               'data/tensorflow_PRs.txt','data/vscode_PRs.txt',
               'data/flutter_PRs.txt']

In [3]:
def get_dataset(loc1,loc2):
    df = pd.read_csv(loc1)
    df2 = pd.read_table(loc2, sep = ',')
    issue_id = df.values[:,1].tolist()
    lbl_lst = df.values[:,3].tolist()
    all_issue = df2.values[:,1].tolist()
    question_lst = df2.values[:,11].tolist()
    title_lst = df2.values[:,7].tolist()

    newdic = dict(zip(issue_id,lbl_lst))
    pos_issue = list(set([i for i,j in newdic.items() if j == 0]))
    all_neg_issue = list(set([x for x in all_issue if x not in issue_id]))
    neg_issue = random.sample(all_neg_issue,len(pos_issue))
    pos_question = [q for t,q,i in zip(title_lst,question_lst,all_issue) if i in pos_issue]
    pos_title = [t for t,q,i in zip(title_lst,question_lst,all_issue) if i in pos_issue]

    neg_question = [q for t,q,i in zip(title_lst,question_lst,all_issue) if i in neg_issue]
    neg_title = [t for t,q,i in zip(title_lst,question_lst,all_issue) if i in neg_issue]

    title_lst = pos_title + neg_title
    question_lst= pos_question + neg_question
    lable_lst = [1 for i in range(len(pos_question))] + [0 for i in range(len(neg_question))]
    
    return title_lst,question_lst,lable_lst


In [4]:
title_list = []
question_list = []
lable_list = []
for loc1,loc2 in zip(pos_locations,q_locations):
    t1,q1,l1 = get_dataset(loc1,loc2)
    title_list +=t1
    question_list+=q1
    lable_list+=l1
    
print(len(title_list))
print(len(question_list))
print(len(lable_list))


29903
29903
29903


In [5]:
labeled_question = pd.DataFrame({'title':title_list,'body': question_list,'label':lable_list})

print(labeled_question.shape)
labeled_question.dropna(axis=0,how='any',inplace=True)
labeled_question['body'] = labeled_question['body'].str.lower()
labeled_question['title'] = labeled_question['title'].str.lower()
print(labeled_question.shape)
labeled_question.head()


(29903, 3)
(29016, 3)


,title,body,label
0,possible to use spring-boot on android develop...,<!-- thanks for raising a spring boot issue. p...,1
1,how do i use a custom websockethandler when i ...,my code is like this ```java @configuration @...,1
3,can i use a higher version of the `spring-boot...,can i use a higher version of the `spring-boot...,1
4,webflux functional endpoints authentication,i only found the documentation about authentic...,1
5,document that the minimum supported version of...,"hi, i deployed a project made with https://sta...",1


In [6]:
labeled_question['label'].value_counts()

1    14676
0    14340
Name: label, dtype: int64

In [7]:
df1 = pd.read_csv(r'Dataset_SO\StackOverflow\vscode_stackoverflow.csv')
df2 = pd.read_csv(r'Dataset_SO\StackOverflow\tensorflow_stackoverflow.csv')
df3 = pd.read_csv(r'Dataset_SO\StackOverflow\spring-boot_stackoverflow.csv')
df4 = pd.read_csv(r'Dataset_SO\StackOverflow\flutter_stackoverflow.csv')

In [8]:
Strong_pos_lst_body = df2.values[:,3].tolist() + df2.values[:,3].tolist() + df2.values[:,3].tolist() + df2.values[:,3].tolist()
Strong_pos_lst_title = df2.values[:,2].tolist() + df2.values[:,2].tolist() + df2.values[:,2].tolist() + df2.values[:,2].tolist()
SOpos_df_all = pd.DataFrame({'title':Strong_pos_lst_title,'body': Strong_pos_lst_body,'label':[1 for i in range(len(Strong_pos_lst_title))]})
SOpos_df = SOpos_df_all.sample(7500,ignore_index=True)
SOpos_df

,title,body,label
0,Kernel dying while importing Tensorflow,<p>In anaconda3 after creating a new environme...,1
1,Permanently update a variable in tensorflow-ja...,<p>I have trained a model using python-tensorf...,1
2,Distributed Tensorflow: good example for synch...,\n\n<p>I am new to distributed tensorflow and ...,1
3,AttributeError: 'str' object has no attribute ...,<p>I was trying to make an Image Captioning mo...,1
4,Keras Multi-GPU: One CPU core goes to 100% and...,<p>I have been using the system with one GTX10...,1
...,...,...,...
7495,How to map a dataset with a function which con...,<p>I have a dataset in the form of a Tensorflo...,1
7496,Is it allowed to assign a value to a variable ...,<p>I define a simple computational graph invol...,1
7497,Placing examples in a mini-batch on the rows o...,<p>I am using tensorflow 2.0 to train a model....,1
7498,Tensorflow padding='SAME' applied before/after...,<p>When using TF's Conv2D layer with <code>pad...,1


In [9]:
df_pos_all = labeled_question.loc[labeled_question['label']==1]
df_neg = labeled_question.loc[labeled_question['label']==0]
df_pos = df_pos_all.sample(7500,ignore_index=True)
frame = [df_pos, SOpos_df, df_neg]

labeled_question = pd.concat(frame, ignore_index=True)
labeled_question['label'].value_counts()

1    15000
0    14340
Name: label, dtype: int64

Data Cleaning

In [10]:
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
import re, string, unicodedata

In [11]:
def html_remover(data):
  beauti = BeautifulSoup(data,'html.parser')
  return beauti.get_text()

# to remove URL
def url_remover(data):
  return re.sub(r'https\S','',data)

def web_associated(data):
  text = html_remover(data)
  text = url_remover(text)
  return text

In [12]:
def remove_round_brackets(data):
  return re.sub('\(.*?\)','',data)

def remove_punc(data):
  trans = str.maketrans('','', string.punctuation)
  return data.translate(trans)

def white_space(data):
  return ' '.join(data.split())

def complete_noise(data):
  new_data = remove_round_brackets(data)
  new_data = remove_punc(new_data)
  new_data = white_space(new_data)
  return new_data

In [13]:
def Data_cleaning(x):
    return complete_noise(web_associated(x))

for index,row in labeled_question.iterrows():
    text = Data_cleaning(row['body'])
    text1 = wordninja.split(text)
    text2 = ' '.join(text1)
    labeled_question.loc[index,'body'] = text2

cleaned_labeled_question = labeled_question.loc[labeled_question['body'].str.len()>10]
cleaned_labeled_question.head()


c:\Users\79105\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\79105\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
c:\Users\79105\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use

,title,body,label
0,tensorflow/lite/kernels/gather.cc:80 0 <= axis...,i have converted a mask r cnn model to deploy ...,1
1,macbook m1 flutter run -v error,steps to reproduce macbook m 1 2020 1 flutter ...,1
2,makefile build symbol(s) not found for archite...,im trying to build tensor flow on a mac os x s...,1
3,crash in _scrollablescope. textdirection!=null...,steps to reproduce hello this is my fir ts cod...,1
4,android toolchain,steps to reproduce 1 2 3 logs logs,1


In [14]:
cleaned_labeled_question.head()

,title,body,label
0,tensorflow/lite/kernels/gather.cc:80 0 <= axis...,i have converted a mask r cnn model to deploy ...,1
1,macbook m1 flutter run -v error,steps to reproduce macbook m 1 2020 1 flutter ...,1
2,makefile build symbol(s) not found for archite...,im trying to build tensor flow on a mac os x s...,1
3,crash in _scrollablescope. textdirection!=null...,steps to reproduce hello this is my fir ts cod...,1
4,android toolchain,steps to reproduce 1 2 3 logs logs,1


In [15]:
cleaned_labeled_question.loc[1,'body']

'steps to reproduce macbook m 1 2020 1 flutter run v logs console dex pods build debug iphone simulator appa u th build all product headers yam l i quote users galland s library developer x code derived data runner dnc aw uml uuo mep gz ew bx bgh tim uf build intermediates no in dex pods build debug iphone simulator appa u th build appa u th project headers h map i users galland s documents church notes app flutter build ios debug iphone simulator appa u th include i users galland s library developer x code derived data runner dnc aw uml uuo mep gz ew bx bgh tim uf build intermediates no index pods build debug iphone simulator appa u th build derived sources normal i 386 i users galland s library developer x code derived data runner dnc aw uml uuo mep gz ew bx bgh tim uf build intermediates no index pods build debug iphone simulator appa u th build derived sources i 386 i users galland s library developer x code derived data runner dnc aw uml uuo mep gz ew bx bgh tim uf build intermedi

In [16]:
cleaned_labeled_question['label'].value_counts()

1    14929
0    14246
Name: label, dtype: int64

Output Data

In [17]:
cleaned_labeled_question.to_csv('4.0labeled_t+b_for_all.csv', index=False, encoding='utf-8')

In [18]:
# testing = cleaned_labeled_question.sample(5000)
# testing.to_csv('4.0testing_t+b.csv', index=False, encoding='utf-8')

In [19]:
small = cleaned_labeled_question.sample(3000)
small.to_csv('4.0labeled_t+b_small_sample.csv', index=False, encoding='utf-8')

In [20]:
small['label'].value_counts()

1    1528
0    1472
Name: label, dtype: int64